In [1]:
%load_ext autoreload
%load_ext dotenv
%autoreload 2
%dotenv

In [14]:
import re

import numpy as np
import pandas as pd
import os
import featuretools as ft
from shl.prepare import normalize_epoch_time, normalize_lat_long
from shl.features import fetch_location, NoLocationFoundException, visualize_trace
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
import time
from tqdm import tqdm
import csv

from shl.features import WifiFeature

In [3]:
train_cell = normalize_epoch_time(pd.read_parquet('../data/train/Cells.parquet'), 'epoch_time')
train_location = normalize_epoch_time(pd.read_parquet('../data/train/Location.parquet'), 'epoch_time')
test_cell = normalize_epoch_time(pd.read_parquet('../data/test/Cells.parquet'), 'epoch_time')
test_location = normalize_epoch_time(pd.read_parquet('../data/test/Location.parquet'), 'epoch_time')
validate_cell = normalize_epoch_time(pd.read_parquet('../data/validate/Cells.parquet'), 'epoch_time')
validate_location = normalize_epoch_time(pd.read_parquet('../data/validate/Location.parquet'), 'epoch_time')

train_label = normalize_epoch_time(pd.read_parquet('../data/train/Label.parquet'), 'epoch_time')
test_label = normalize_epoch_time(pd.read_parquet('../data/test/Label_idx.parquet'), 'epoch_time')
validate_label = normalize_epoch_time(pd.read_parquet('../data/validate/Label.parquet'), 'epoch_time')

In [4]:
fetch_location(train_cell.query('epoch_time == 1490448212563'))

{'Latitude': 51.4723711, 'Longitude': -0.4135758, 'accuracy': 873}

In [5]:
def fetch_cell_with_missing_location(cell: pd.DataFrame, location: pd.DataFrame) -> pd.DataFrame:
    return cell.merge(location, on='epoch_time_id', how="left").sort_values(by='epoch_time_id').query('accuracy != accuracy and cell_type == cell_type')

print(fetch_cell_with_missing_location(train_cell, train_location)['epoch_time_id'].value_counts().shape)
print(fetch_cell_with_missing_location(validate_cell, validate_location)['epoch_time_id'].value_counts().shape)
print(fetch_cell_with_missing_location(test_cell, test_location)['epoch_time_id'].value_counts().shape)

(424308,)
(47129,)
(165563,)


In [20]:
train_cell_with_missing_location = fetch_cell_with_missing_location(train_cell, train_location)
train_cell_with_missing_location

,epoch_time_x,num_entries,cell_type,isRegistered,ci,MCC,MNC,PCI,TAC,asuLevel,...,lac,PSC,epoch_time_id,epoch_time_y,Ignore1,Ignore2,accuracy,Latitude,Longitude,Altitude
0,1490430923343,3,LTE,1,1.280004e+08,234,10,20.0,1.440000e+02,25,...,NaN,NaN,1490430923000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1490430923343,3,LTE,0,2.147484e+09,234,10,398.0,2.147484e+09,22,...,NaN,NaN,1490430923000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1490430923343,3,LTE,0,2.147484e+09,234,10,4.0,2.147484e+09,16,...,NaN,NaN,1490430923000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1490430923349,3,LTE,1,1.280004e+08,234,10,20.0,1.440000e+02,25,...,NaN,NaN,1490430923000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1490430923349,3,LTE,0,2.147484e+09,234,10,398.0,2.147484e+09,22,...,NaN,NaN,1490430923000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040562,1499268459625,11,LTE,0,2.147484e+09,234,10,398.0,2.147484e+09,23,...,NaN,NaN,1499268460000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3040563,1499268459625,11,LTE,0,2.147484e+09,234,10,20.0,2.147484e+09,13,...,NaN,NaN,1499268460000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4481215,1499268459625,11,GSM,0,NaN,234,10,NaN,NaN,2,...,2.147484e+09,NaN,1499268460000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3040565,1499268460645,2,LTE,0,2.147484e+09,234,10,398.0,2.147484e+09,18,...,NaN,NaN,1499268461000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
train_cell_with_missing_location[train_cell_with_missing_location['epoch_time_id'].diff() == 1000]

,epoch_time_x,num_entries,cell_type,isRegistered,ci,MCC,MNC,PCI,TAC,asuLevel,...,lac,PSC,epoch_time_id,epoch_time_y,Ignore1,Ignore2,accuracy,Latitude,Longitude,Altitude
17,1490430960168,3,LTE,0,2.147484e+09,234,10,398.0,2.147484e+09,22,...,NaN,NaN,1490430960000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,1490604351223,5,LTE,0,2.147484e+09,234,10,398.0,2.147484e+09,17,...,NaN,NaN,1490604351000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,1490604352104,5,LTE,0,2.147484e+09,234,10,34.0,2.147484e+09,16,...,NaN,NaN,1490604352000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,1490604353155,5,LTE,1,1.280004e+08,234,10,20.0,1.440000e+02,27,...,NaN,NaN,1490604353000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,1490688592844,1,LTE,1,1.280004e+08,234,10,20.0,1.440000e+02,27,...,NaN,NaN,1490688593000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040554,1499268456589,2,LTE,1,1.280004e+08,234,10,20.0,1.440000e+02,25,...,NaN,NaN,1499268457000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3040556,1499268457596,2,LTE,1,1.280004e+08,234,10,20.0,1.440000e+02,25,...,NaN,NaN,1499268458000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4481205,1499268458618,11,GSM,0,NaN,234,10,NaN,NaN,8,...,2.147484e+09,NaN,1499268459000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4481211,1499268459625,11,GSM,0,NaN,234,10,NaN,NaN,8,...,2.147484e+09,NaN,1499268460000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(train_location['epoch_time_id'].value_counts().shape)
print(test_location['epoch_time_id'].value_counts().shape)
print(validate_location['epoch_time_id'].value_counts().shape)

(908631,)
(560792,)
(101228,)


In [8]:
def convert_cell_to_location(cells_data, location_data, file_path: str, step: int = 60):
    with open(file_path, "w") as file:
        csv_writer = csv.DictWriter(file, fieldnames=['epoch_time_id', 'Latitude', 'Longitude', 'accuracy'])
        csv_writer.writeheader()
        cells_groups = fetch_cell_with_missing_location(cells_data, location_data).groupby('epoch_time_id')
        for i, (epoch_time, cells) in enumerate(tqdm(cells_groups, total=len(cells_groups))):
            if i % step != 0:
                continue
            try:
                csv_writer.writerow({'epoch_time_id': epoch_time, **fetch_location(cells)})
            except NoLocationFoundException as e:
                print(f'Missing location for:\n{e.cells}')
                continue
            finally:
                time.sleep(0.5)
            file.flush()

convert_cell_to_location(train_cell, train_location, '../data/train/features_cell_to_location.csv', step=60)
convert_cell_to_location(test_cell, test_location, '../data/test/features_cell_to_location.csv', step=60)
convert_cell_to_location(validate_cell, validate_location, '../data/validate/features_cell_to_location.csv', step=60)



  3%|▎         | 12961/424308 [02:36<1:28:01, 77.89it/s]

Missing location for:
{'cellTowers': [{'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}]}


  3%|▎         | 13081/424308 [02:37<1:19:32, 86.17it/s]

Missing location for:
{'cellTowers': [{'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}]}


  3%|▎         | 14401/424308 [02:53<1:20:44, 84.61it/s]

...

 74%|███████▎  | 34681/47129 [07:37<02:24, 85.97it/s]

Missing location for:
{'cellTowers': [{'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}]}


 79%|███████▉  | 37321/47129 [08:08<01:58, 82.72it/s]

Missing location for:
{'cellTowers': [{'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}, {'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}, {'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}, {'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}, {'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}, {'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}, {'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}, {'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}, {'cellId': 2147483647, 'locationAreaCode': 2147483647, 'mobileCountryCode': 234, 'mobileNetworkCode': 10}]

100%|██████████| 47129/47129 [10:06<00:00, 77.67it/s]


In [24]:
train_cell_location = pd.read_csv('../data/train/features_cell_to_location.csv')
display(visualize_trace(train_label.merge(train_cell, on="epoch_time_id").merge(train_cell_location, on="epoch_time_id")))

test_cell_location = pd.read_csv('../data/test/features_cell_to_location.csv')
display(visualize_trace(test_label.merge(test_cell, on="epoch_time_id").merge(test_cell_location.assign(label=lambda x: 1), on="epoch_time_id")))

In [23]:
train_cell_with_missing_location[train_cell_with_missing_location['epoch_time_id'].diff() == 1000][['epoch_time_id']].merge(train_cell_location, on='epoch_time_id', how='left')

,epoch_time_id,Latitude,Longitude,accuracy
0,1490430960000,NaN,NaN,NaN
1,1490604351000,NaN,NaN,NaN
2,1490604352000,NaN,NaN,NaN
3,1490604353000,NaN,NaN,NaN
4,1490688593000,NaN,NaN,NaN
...,...,...,...,...
405745,1499268457000,NaN,NaN,NaN
405746,1499268458000,NaN,NaN,NaN
405747,1499268459000,NaN,NaN,NaN
405748,1499268460000,NaN,NaN,NaN


In [18]:
train_cell.query('epoch_time == 1490448212563')

,epoch_time,num_entries,cell_type,isRegistered,ci,MCC,MNC,PCI,TAC,asuLevel,dBm,level,cid,lac,PSC,epoch_time_id
89,1490448212563,2,LTE,1,1.280593e+08,234,10,425.0,4.096000e+03,41,-99,3,NaN,NaN,NaN,1490448213000
90,1490448212563,2,LTE,0,2.147484e+09,234,10,424.0,2.147484e+09,39,-101,3,NaN,NaN,NaN,1490448213000


In [24]:
train_cell['ci'].astype('int64', errors='ignore')

0          1.280004e+08
1          2.147484e+09
2          2.147484e+09
3          1.280004e+08
4          2.147484e+09
               ...     
4474375             NaN
4474376             NaN
4474377             NaN
4474378             NaN
4474379             NaN
Name: ci, Length: 4474380, dtype: float64

In [3]:
validate_cell_with_label = validate_label.merge(train_cell, how='left', on='epoch_time_id').sort_values(['epoch_time_id', 'cell_type', 'dBm'])[['epoch_time_id', 'label', 'cell_type', 'dBm', 'asuLevel', 'ci']]

#.loc[:, ['epoch_time', 'label', 'SSID', 'RSSI']].sort_values(['epoch_time', 'RSSI'], ascending=[True, False]).groupby('epoch_time').agg({'label' : 'first', 'SSID' : 'first'}).reset_index()

In [10]:
validate_cell_with_label.to_csv('./validate_cell_with_label.csv')